In [50]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [51]:
plot_cols = ['ENTRY', 'STOPLOSS', 'TAKEPROFIT']
plot_colours = ['#043ef9', '#eb5334', '#34eb37']
def plot_candles(df_plot):
    fig = go.Figure()
    fig.add_trace(go.Candlestick(
        x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
        line=dict(width=1), opacity=1,
        increasing_fillcolor='#24A06B',
        decreasing_fillcolor="#CC2E3C",
        increasing_line_color='#2EC886',  
        decreasing_line_color='#FF3A4C'
    ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_buys.time,
            y=df_buys[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colours[i],size=12)
        ))
    for i in range(0, 3):
        fig.add_trace(go.Scatter(
            x=df_sells.time,
            y=df_sells[plot_cols[i]],
            mode='markers',
            marker=dict(color=plot_colours[i],size=12)
        ))
    fig.update_layout(width=1000,height=400,
        margin=dict(l=10,r=10,b=10,t=10),
        font=dict(size=10,color="#e1e1e1"),
        paper_bgcolor="#1e1e1e",
        plot_bgcolor="#1e1e1e")
    fig.update_xaxes(
        gridcolor="#1f292f",
        showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
        rangebreaks=[
            dict(bounds=["sat", "mon"])
        ]
    )
    fig.update_yaxes(
        gridcolor="#1f292f",
        showgrid=True
    )
    fig.show()

In [52]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [53]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [54]:
df_raw.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2022-01-02T22:00:00.000000000Z,4251,115.118,115.294,115.084,115.213,115.068,115.286,115.047,115.206,115.168,115.301,115.107,115.220
1,2022-01-03T02:00:00.000000000Z,3076,115.214,115.318,115.207,115.312,115.207,115.311,115.200,115.304,115.221,115.324,115.213,115.319
2,2022-01-03T06:00:00.000000000Z,7666,115.310,115.364,115.150,115.178,115.304,115.356,115.144,115.171,115.317,115.371,115.156,115.184
3,2022-01-03T10:00:00.000000000Z,11488,115.176,115.179,114.950,115.149,115.170,115.172,114.941,115.142,115.183,115.186,114.957,115.156
4,2022-01-03T14:00:00.000000000Z,15073,115.150,115.370,115.138,115.320,115.144,115.363,115.132,115.314,115.157,115.377,115.145,115.326


In [55]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.ask_h_prev
    elif row.SIGNAL == -1:
        return row.bid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0

def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.ENTRY - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.ENTRY + (row.RANGE_prev * SLOSS)
    else:
        return 0


def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.ENTRY + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.ENTRY - (row.RANGE_prev * TPROFIT)
    else:
        return 0

In [56]:
df = df_raw.copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['ask_h_prev'] = df.ask_h.shift(1)
df['bid_l_prev'] = df.bid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)

In [57]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

In [58]:
df[df.SIGNAL!=0].to_pickle("USD_JPY_H4_trades.pkl")

In [59]:
class Trade():
    def __init__(self, row):
        self.candle_date = row.time
        self.direction = row.SIGNAL
        self.entry = row.ENTRY
        self.TP = row.TAKEPROFIT
        self.SL = row.STOPLOSS
        self.running = False
        self.result = None
        self.stopped = None

    def update(self, row):
        if self.running == True:
            self.update_result(row)
        else:
            self.check_entry(row)    
    
    def check_entry(self, row):
        if self.direction == 1 and row.mid_c >= self.entry or self.direction == -1 and row.mid_c <= self.entry:
            self.index = row.name
            self.opened = row.time
            self.running = True

    def update_result(self, row):
        if self.direction == 1:
            if row.mid_c >= self.TP:
                self.result = 2.0
            elif row.mid_c <= self.SL:
                self.result = -1.0
        else:
            if row.mid_c <= self.TP:
                self.result = 2.0            
            elif row.mid_c >= self.SL:
                self.result = -1.0   
        
        if self.result is not None:
            self.running = False
            self.stopped = row.time


In [60]:
open_trades = []
closed_trades = []

for index, row in df.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.stopped is not None:
            closed_trades.append(ot)
    
    open_trades = [x for x in open_trades if x.stopped is None]

    if row.SIGNAL != 0:
        open_trades = [x for x in open_trades if x.running == True]
        open_trades.append(Trade(row))


In [61]:
df_trades = pd.DataFrame.from_dict([vars(x) for x in closed_trades])

In [62]:
df[df.SIGNAL!=0].head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,...,mid_l_prev,ask_h_prev,bid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
4,2022-01-03T18:00:00.000000000Z,4419,115.318,115.358,115.263,115.350,115.312,115.350,115.256,115.343,...,115.138,115.377,115.132,0.232,1,1,1,115.4002,115.3074,115.5858
9,2022-01-04T14:00:00.000000000Z,17583,116.260,116.338,115.955,116.173,116.253,116.331,115.948,116.166,...,115.811,116.359,115.804,0.541,-1,1,1,116.4131,116.1967,116.8459
10,2022-01-04T18:00:00.000000000Z,5432,116.172,116.184,116.080,116.148,116.164,116.177,116.074,116.137,...,115.955,116.345,115.948,0.383,-1,-1,-1,115.9097,116.0629,115.6033
20,2022-01-06T10:00:00.000000000Z,14994,115.750,115.892,115.697,115.750,115.743,115.885,115.690,115.745,...,115.658,115.963,115.652,0.298,-1,-1,-1,115.6222,115.7414,115.3838
24,2022-01-07T02:00:00.000000000Z,6340,115.898,115.939,115.833,115.914,115.891,115.931,115.827,115.908,...,115.830,116.056,115.785,0.218,1,1,1,116.0778,115.9906,116.2522


In [63]:
df_plot = df.iloc[0:60]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]

In [64]:
df_trades.result.sum()

191.0

In [65]:
plot_candles(df_plot)